In [449]:
using Random
using Distributions

In [475]:
abstract type AbstractBlk end

struct Minor
    label::String
end

struct Genesis <: AbstractBlk end

struct Blk <: AbstractBlk
    index::Int
    minor::Minor
    prev::AbstractBlk
end

function Base.show(io::IO, b::Genesis)
    Base.show(io, 0)
end

function Base.show(io::IO, b::Blk)
    Base.show(io, (b.index, b.minor.label))
end

struct Queue
    sender::Minor
    receiver::Minor
    elem::Vector{AbstractBlk}
    
    Queue(sender, receiver) = new(sender, receiver, Vector{AbstractBlk}())
end

In [476]:
getid(b::Blk) = b.index
getid(b::Genesis) = 0

getid (generic function with 2 methods)

In [477]:
getreceiver(q::Queue) = q.receiver

function receiveblk!(q::Queue, i::Int)::AbstractBlk
    b = q.elem[i]
    deleteat!(q.elem, i)
    b
end

getlength(q::Queue) = length(q.elem)

getlength (generic function with 1 method)

In [478]:
function sendblk!(q::Queue, b::AbstractBlk)
    push!(q.elem, b)
end

sendblk! (generic function with 1 method)

In [479]:
function event_genBlk!(x::Minor, workblk::Dict{Minor,AbstractBlk}, queue::Vector{Queue}, blocks::Vector{Blk})
    prev = workblk[x]
    id = getid(prev) + 1
    b = Blk(id, x, prev)
    push!(blocks, b)
    workblk[x] = b
    for q = queue
        sendblk!(q, b)
    end
    b
end

event_genBlk! (generic function with 1 method)

In [480]:
function event_arrivalBlk!(q::Queue, i::Int, workblk::Dict{Minor,AbstractBlk})
    x = getreceiver(q)
    b = receiveblk!(q, i)
    prev = workblk[x]
    if getid(prev) < getid(b)
        workblk[x] = b
    end
    b
end

event_arrivalBlk! (generic function with 1 method)

In [481]:
function rates(u1, u2, queue, lambda, mu)
#    u1 = sort(collect(keys(lambda)), by=x->x.label)
    r1 = [lambda[x] for x = u1]
#    u2 = sort(collect(keys(mu)), by=x->(x[1].label, x[2].label))
    r2 = [mu[x] * getlength(queue[x]) for x = u2]
    return r1, r2
end

rates (generic function with 1 method)

In [482]:
a = Minor("A")
b = Minor("B")

Minor("B")

In [509]:
lambda = Dict(
    a => 1/10,
    b => 1/0.1,
)

mu = Dict(
    (a,b) => 1/1,
    (b,a) => 1/10
)

Dict{Tuple{Minor,Minor},Float64} with 2 entries:
  (Minor("B"), Minor("A")) => 0.1
  (Minor("A"), Minor("B")) => 1.0

In [510]:
begin
    rng = MersenneTwister(1234)
    blocks = Vector{Blk}()

    events = 0
    u1 = sort(collect(keys(lambda)), by=x->x.label)
    u2 = sort(collect(keys(mu)), by=x->(x[1].label, x[2].label))
    workblk = Dict{Minor,AbstractBlk}(a=>Genesis(), b=>Genesis())
    qAtoB = Queue(a, b)
    qBtoA = Queue(b, a)
    queue = Dict(
        (a,b)=>qAtoB,
        (b,a)=>qBtoA,
    )
    input = Dict(
        a=>[qAtoB],
        b=>[qBtoA],
    )

    ctime = 0.0
    while ctime < 60*24 # simulation time(1 week)
        r1, r2 = rates(u1, u2, queue, lambda, mu)
        etime1 = [(r != 0) ? rand(rng, Exponential(1/r)) : Inf for r = r1]
        etime2 = [(r != 0) ? rand(rng, Exponential(1/r)) : Inf for r = r2]
        m1, i1 = findmin(etime1)
        m2, i2 = findmin(etime2)
        if m1 < m2
            ctime += m1
            event_genBlk!(u1[i1], workblk, input[u1[i1]], blocks)
        else
            ctime += m2
            v = rand(rng, DiscreteUniform(1,getlength(queue[u2[i2]])))
            event_arrivalBlk!(queue[u2[i2]], v, workblk)
        end
        events += 1
    end
end

In [511]:
@time workblk

  0.000001 seconds (3 allocations: 144 bytes)


Dict{Minor,AbstractBlk} with 2 entries:
  Minor("A") => (14501, "B")
  Minor("B") => (14522, "B")

In [512]:
workblk[b].prev

(14521, "B")

In [513]:
workblk[a].prev

(14500, "B")

In [514]:
blk = workblk[a]
chains = []
push!(chains, blk)
while blk != Genesis()
    blk = blk.prev
    push!(chains, blk)
end

In [515]:
chains

14502-element Array{Any,1}:
 (14501, "B")
 (14500, "B")
 (14499, "B")
 (14498, "B")
 (14497, "B")
 (14496, "B")
 (14495, "B")
 (14494, "B")
 (14493, "B")
 (14492, "B")
 (14491, "B")
 (14490, "B")
 (14489, "B")
 ⋮           
 (11, "B")   
 (10, "B")   
 (9, "B")    
 (8, "B")    
 (7, "B")    
 (6, "B")    
 (5, "B")    
 (4, "B")    
 (3, "B")    
 (2, "B")    
 (1, "B")    
 0           

In [516]:
blocks

14664-element Array{Blk,1}:
 (1, "B")    
 (2, "B")    
 (3, "B")    
 (4, "B")    
 (5, "B")    
 (6, "B")    
 (7, "B")    
 (8, "B")    
 (9, "B")    
 (10, "B")   
 (11, "B")   
 (12, "B")   
 (13, "B")   
 ⋮           
 (14511, "B")
 (14512, "B")
 (14513, "B")
 (14514, "B")
 (14515, "B")
 (14516, "B")
 (14517, "B")
 (14518, "B")
 (14519, "B")
 (14520, "B")
 (14521, "B")
 (14522, "B")

In [517]:
invchains = reverse(chains)

14502-element Array{Any,1}:
 0           
 (1, "B")    
 (2, "B")    
 (3, "B")    
 (4, "B")    
 (5, "B")    
 (6, "B")    
 (7, "B")    
 (8, "B")    
 (9, "B")    
 (10, "B")   
 (11, "B")   
 (12, "B")   
 ⋮           
 (14490, "B")
 (14491, "B")
 (14492, "B")
 (14493, "B")
 (14494, "B")
 (14495, "B")
 (14496, "B")
 (14497, "B")
 (14498, "B")
 (14499, "B")
 (14500, "B")
 (14501, "B")

In [518]:
open("chains.txt","w") do out #ファイルに書く
    print(out,invchains)
end

In [519]:
open("blocks.txt","w") do out #ファイルに書く
    print(out,blocks)
end

In [435]:
haki = 0
for i in 2:length(invchains)
    if invchains[i].minor == Minor("B")
        for j in 1:length(blocks)
            if (blocks[j].minor == Minor("A")) && (blocks[j].index == invchains[i].index)
                haki += 1
            end
        end
    end
end
haki

366

In [436]:
print("chains:",length(chains),"   blocks:",length(blocks),"   haki:",haki)

chains:11819   blocks:13038   haki:366